In [ ]:
!pip install indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 799.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 10.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt

import torch
from torch import nn,tensor
from torch.utils.data import Dataset,DataLoader
from torchtext.vocab import build_vocab_from_iterator
from tqdm import tqdm
import re
import numpy as np
import pandas as pd
import pickle
from indicnlp.tokenize import indic_tokenize
import seaborn as sns
import copy

train_path = '/content/drive/MyDrive/Colab Notebooks/INLP Project/product_reviews/hi/hi-train.csv'
test_path ='/content/drive/MyDrive/Colab Notebooks/INLP Project/product_reviews/hi/hi-test.csv'
syn_path = '/content/drive/MyDrive/Colab Notebooks/INLP Project/product_reviews/hi/hi-test-updated.csv'
val_path = '/content/drive/MyDrive/Colab Notebooks/INLP Project/product_reviews/hi/hi-valid.csv'
forward_model_path ='/content/drive/MyDrive/Colab Notebooks/INLP Project/train embeddings/pretrained_forward_model'
backward_model_path ='/content/drive/MyDrive/Colab Notebooks/INLP Project/train embeddings/pretrained_backward_model'
word_to_ix_path = '/content/drive/MyDrive/Colab Notebooks/INLP Project/train embeddings/word_to_ix.pkl'

In [ ]:

train_df = pd.read_csv(train_path, header=None, names=['Label', 'Text'])

test_df = pd.read_csv(test_path, header=None, names=['Label', 'Text'])
syn_df = pd.read_csv(syn_path, header=None, names=['Label', 'Text'])
test_df.head()



,Label,Text
0,neutral,"मगर , इस तरफ कोई ध्यान नहीं दिया जा रहा ।"
1,negative,गेम कई बार मुश्किल मालूम देता है ।
2,negative,मल्टीप्लेयर में छोटी स्टोरीलाइन है ।
3,positive,हाई - एंड एचपी नोटबुक्स की लुक्स हमेशा बेहतरीन...
4,positive,कैमरे बनाने वाली कंपनी निकोन अपनी 1 सीरीज कैमर...


In [ ]:
syn_df.head()


,Label,Text
0,neutral,"मगर , इस तरफ कोई नज़र नहीं दी जा रही ।"
1,negative,गेम कई बार कठिन मालूम देता है ।
2,negative,मल्टीप्लेयर में लघु स्टोरीलाइन है ।
3,positive,हाई - एंड एचपी नोटबुक्स की लुक्स हमेशा उत्तम ह...
4,positive,कैमरे बनाने वाली कंपनी निकोन अपनी 1 सीरीज कैमर...


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_df['Label'] = label_encoder.fit_transform(train_df['Label'])
test_df['Label'] = label_encoder.transform(test_df['Label'])
syn_df['Label'] = label_encoder.transform(syn_df['Label'])

test_df.head()


,Label,Text
0,1,"मगर , इस तरफ कोई ध्यान नहीं दिया जा रहा ।"
1,0,गेम कई बार मुश्किल मालूम देता है ।
2,0,मल्टीप्लेयर में छोटी स्टोरीलाइन है ।
3,2,हाई - एंड एचपी नोटबुक्स की लुक्स हमेशा बेहतरीन...
4,2,कैमरे बनाने वाली कंपनी निकोन अपनी 1 सीरीज कैमर...


In [ ]:
syn_df.head()


,Label,Text
0,1,"मगर , इस तरफ कोई नज़र नहीं दी जा रही ।"
1,0,गेम कई बार कठिन मालूम देता है ।
2,0,मल्टीप्लेयर में लघु स्टोरीलाइन है ।
3,2,हाई - एंड एचपी नोटबुक्स की लुक्स हमेशा उत्तम ह...
4,2,कैमरे बनाने वाली कंपनी निकोन अपनी 1 सीरीज कैमर...


In [ ]:
class ElmoProcessor(torch.nn.Module):
    def __init__(self,vocab_size,embedding_dim,hidden_dim,wt_mat):
        super(ElmoProcessor,self).__init__()
        #self.embeddings=nn.Embedding(vocab_size,embedding_dim)
        self.lstmf1=nn.LSTM(embedding_dim,hidden_dim,num_layers=1,bidirectional=False,batch_first=True)
        self.lstmf2=nn.LSTM(hidden_dim,hidden_size=hidden_dim,bidirectional=False,batch_first=True)
        self.embedding=wt_mat.to(device)
        # self.lstmb1=nn.LSTM(embedding_dim,hidden_dim,num_layers=1,bidirectional=False,batch_first=True)
        # self.lstmb2=nn.LSTM(hidden_dim,hidden_size=hidden_dim,bidirectional=False,batch_first=True)

        self.linear=nn.Linear(hidden_dim,vocab_size)
    def forward(self,embeddingdata):
        emb=torch.tensor(self.embedding(embeddingdata.to(device)))

        emb_f=emb
        # emb_b=emb.flip(1)

        o1f1,h1f1=self.lstmf1(emb_f)
        o2f2,h2f2=self.lstmf2(o1f1)
        # o2f2=o2f2+o1f1
        # h2f2=h2f2+h1f1
        # o1b1,h1b1=self.lstmb1(emb_b)
        # o2b2,h2b2=self.lstmb2(o1b1,h1b1)
        # o2b2=o2b2+o1b1
        # h2b2=h2b2+h1b1

        # weighted_vector_emb=torch.cat([emb_f,emb_b],dim=2)
        # weighted_vector_h1=torch.cat([o1f1,o1f1],dim=2)
        # weighted_vector_h2=torch.cat([o1f1,o2f2],dim=2)
        output =self.linear(o2f2)
        return (output,o1f1,o2f2)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
# # not run

# wt_mat=torch.zeros(size=(90337,300))
# embeddings=torch.tensor(wt_mat,dtype=torch.float)
# forward_model= torch.load(forward_model_path,map_location=torch.device('cpu'))
# backward_model= torch.load(backward_model_path,map_location=torch.device('cpu'))

In [ ]:
with open(word_to_ix_path, 'rb') as file:

    word_to_ix = pickle.load(file)


In [ ]:
# # not run
# mxlen=[]
# for column in train_df[train_df.columns[1]]:
#     mxlen.append(len(column.split(' ')))
# max(mxlen)

In [ ]:

mxlen = 30
class DatafetchClassify(Dataset):
    def __init__(self,split:str):
        sequence=[]
        labels=[]
        for x in tqdm(split[split.columns[1]],desc='Generate next word prediction data'):
            if len(x)<2 or x[0]=='=':continue
            words=indic_tokenize.trivial_tokenize(x,'hi') # alternative: x.strip().split(' ')
            words=words
            indices=[word_to_ix.get(word, 1) for word in words[:mxlen]]
            sequence.append((mxlen-len(indices))*[word_to_ix['<PAD>']]+indices[:])
        for x in tqdm(split[split.columns[0]],desc="Generate label"):
            labels.append(x)
        max_seq_len=max([len(ngram) for ngram in sequence])
        self.sequence=tensor(sequence)
        self.sequence.to(device)
        self.labels=tensor(labels)
        self.labels.to(device)
    def __len__(self)->int:
        return len(self.sequence)
    def __getitem__(self,index:int):
        return self.sequence[index],self.labels[index]

In [ ]:
# # not run
# classifier_dataset=DatafetchClassify(train_df)

In [ ]:
batch_size=64

In [ ]:
# # not run
# classifer_dataloader=DataLoader(classifier_dataset,batch_size,shuffle=True)
# data,labels=next(iter(classifer_dataloader))


In [ ]:
class ElmoClassifier(torch.nn.Module):
    def __init__(self,no_classes,forward_model,backward_model,embedding_dim):
        super(ElmoClassifier,self).__init__()
        self.forward_model=forward_model
        self.embedding = forward_model.embedding
        self.backward_model=backward_model
        self.forward_model.requires_grad=False
        self.backward_model.requires_grad=False
        self.linear1=nn.Linear(100,no_classes)
        self.param0=nn.Parameter(torch.rand(1),requires_grad=False)
        self.param1=nn.Parameter(torch.rand(1),requires_grad=False)
        self.param2=nn.Parameter(torch.rand(1),requires_grad=False)
        self.bilstm=nn.LSTM(input_size=embedding_dim*2,hidden_size=50,bidirectional=True,batch_first=True)
    def forward(self,embdata):
        _,o1f1,o2f2=self.forward_model(embdata)
        _,o1b1,o2b2=self.backward_model(embdata)
        # print(embdata.shape)
        # print('\n',o1f1.shape)
        embdata = self.embedding(embdata)
        e = torch.cat([embdata, embdata.flip(1)],dim=2)
        h1=torch.cat([o1f1,o1b1.flip(1)],dim=2)
        h2=torch.cat([o2f2,o2b2.flip(1)],dim=2)
        bio,bih=self.bilstm(self.param0 * e + self.param1 * h1 + self.param2 * h2 )

        y=self.linear1(bio[:,-1,:])

        return y

In [ ]:
# # not run
# classifier=ElmoClassifier(3,forward_model,backward_model,300)

In [ ]:
# # not run

# for epoch in range(5):
#     train_loss=0
#     classifier.train()
#     for batch in tqdm(classifer_dataloader,desc="Training"):
#         x_train,y_train=batch
#         optimizer.zero_grad()

#         x_train.to(device)
#         y_train.to(device)
#         output=classifier(x_train)
#         loss=loss_fn(output.to(device),y_train.to(device))
#         loss.backward()
#         optimizer.step()
#         train_loss +=loss.item()
#     print(train_loss)

In [ ]:
classifier_eval=DatafetchClassify(test_df)

Generate label: 100%|██████████| 523/523 [00:00<00:00, 920915.61it/s]


In [ ]:
classifer_dataloader_eval=DataLoader(classifier_eval,batch_size=100,shuffle=None)
data,labels=next(iter(classifer_dataloader_eval))


In [ ]:
classifier_path = '/content/drive/MyDrive/Colab Notebooks/INLP Project/product_reviews/product_review_classifier_lambdas_frozen'
classifier = torch.load(classifier_path, map_location=torch.device('cpu'))

classifier.to(device)
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(classifier.parameters(),1e-3)

In [ ]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score,recall_score
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
confusion_matrix=np.zeros((4,4))
y_true=[]
y_pred=[]
train_loss=0.0
for batch in tqdm(classifer_dataloader_eval,desc="Evaluation"):
    x_train,y_train=batch
    optimizer.zero_grad()

    x_train.to(device)
    y_train.to(device)
    output=classifier(x_train)

    _,preds=torch.max(output,dim=1)
    y_true.extend(y_train.cpu().detach().numpy())
    y_pred.extend(preds.cpu().detach().numpy())
    break

Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]<ipython-input-8-b766623ffbb4>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb=torch.tensor(self.embedding(embeddingdata.to(device)))
Evaluation:   0%|          | 0/6 [00:17<?, ?it/s]


In [ ]:
output.shape

torch.Size([100, 3])

In [ ]:
len(y_true)

100

In [ ]:
output_exp = torch.exp(output)
row_sums = torch.sum(output_exp, dim=1, keepdim=True)

# Divide each element by its row sum
output = output_exp / row_sums

In [ ]:
y_org_sentence = copy.copy(output)
y_org_sentence

tensor([[0.3314, 0.4841, 0.1845],
        [0.6623, 0.0882, 0.2496],
        [0.1652, 0.4253, 0.4095],
        [0.0027, 0.0057, 0.9917],
        [0.0029, 0.0044, 0.9926],
        [0.0062, 0.0130, 0.9808],
        [0.0199, 0.7848, 0.1953],
        [0.2339, 0.5735, 0.1926],
        [0.3860, 0.1265, 0.4875],
        [0.0035, 0.9697, 0.0268],
        [0.0215, 0.9699, 0.0086],
        [0.0093, 0.6570, 0.3338],
        [0.0047, 0.0533, 0.9420],
        [0.0056, 0.0288, 0.9656],
        [0.0315, 0.1663, 0.8022],
        [0.0459, 0.2235, 0.7306],
        [0.0874, 0.6865, 0.2260],
        [0.0034, 0.9802, 0.0164],
        [0.2586, 0.2663, 0.4751],
        [0.0086, 0.3652, 0.6263],
        [0.6168, 0.2242, 0.1590],
        [0.0066, 0.9577, 0.0357],
        [0.0067, 0.9496, 0.0437],
        [0.6300, 0.2163, 0.1537],
        [0.0084, 0.0145, 0.9770],
        [0.0374, 0.6482, 0.3144],
        [0.1290, 0.7636, 0.1074],
        [0.0693, 0.1495, 0.7812],
        [0.1314, 0.5056, 0.3630],
        [0.650

In [ ]:
# accuracy_score(y_true,y_pred)

In [ ]:
classifier_eval_syn =DatafetchClassify(syn_df)

Generate label: 100%|██████████| 523/523 [00:00<00:00, 1058697.39it/s]


In [ ]:
classifer_dataloader_eval_syn =DataLoader(classifier_eval_syn,batch_size=100,shuffle=None)
data,labels=next(iter(classifer_dataloader_eval_syn))


In [ ]:
# classifier_path = '/content/drive/MyDrive/Colab Notebooks/INLP Project/product_reviews/product_review_classifier_final'
# classifier = torch.load(classifier_path, map_location=torch.device('cpu'))

# classifier.to(device)
# loss_fn=nn.CrossEntropyLoss()
# optimizer=torch.optim.Adam(classifier.parameters(),1e-3)

In [ ]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score,recall_score
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
confusion_matrix=np.zeros((4,4))
y_true=[]
y_pred=[]
train_loss=0.0
for batch in tqdm(classifer_dataloader_eval_syn,desc="Evaluation"):
    x_train,y_train=batch
    optimizer.zero_grad()

    x_train.to(device)
    y_train.to(device)
    output=classifier(x_train)

    _,preds=torch.max(output,dim=1)
    y_true.extend(y_train.cpu().detach().numpy())
    y_pred.extend(preds.cpu().detach().numpy())
    break

Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]<ipython-input-8-b766623ffbb4>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb=torch.tensor(self.embedding(embeddingdata.to(device)))
Evaluation:   0%|          | 0/6 [00:07<?, ?it/s]


In [ ]:
# accuracy_score(y_true,y_pred)

In [ ]:
output.shape

torch.Size([100, 3])

In [ ]:
output_exp = torch.exp(output)
row_sums = torch.sum(output_exp, dim=1, keepdim=True)

# Divide each element by its row sum
output = output_exp / row_sums

In [ ]:
y_syn_sentence = copy.copy(output)
y_syn_sentence

tensor([[0.0208, 0.9625, 0.0167],
        [0.6179, 0.0898, 0.2923],
        [0.0427, 0.6927, 0.2645],
        [0.0028, 0.0062, 0.9910],
        [0.0045, 0.0124, 0.9831],
        [0.0065, 0.0143, 0.9792],
        [0.1286, 0.6369, 0.2345],
        [0.2915, 0.4893, 0.2192],
        [0.4482, 0.1509, 0.4010],
        [0.0037, 0.9605, 0.0358],
        [0.0212, 0.9702, 0.0086],
        [0.0167, 0.2855, 0.6978],
        [0.0070, 0.2413, 0.7518],
        [0.0270, 0.3616, 0.6114],
        [0.0212, 0.0295, 0.9492],
        [0.0697, 0.2468, 0.6835],
        [0.0126, 0.6158, 0.3716],
        [0.0035, 0.9815, 0.0150],
        [0.1436, 0.1811, 0.6753],
        [0.0070, 0.2873, 0.7057],
        [0.5866, 0.2790, 0.1344],
        [0.0061, 0.9600, 0.0339],
        [0.0067, 0.9602, 0.0331],
        [0.4176, 0.5076, 0.0748],
        [0.0110, 0.0229, 0.9662],
        [0.0335, 0.4560, 0.5105],
        [0.1285, 0.7728, 0.0987],
        [0.0889, 0.1762, 0.7349],
        [0.3163, 0.2281, 0.4556],
        [0.381

In [ ]:
import statistics
def evaluate_embeddings(y_org_sentence, y_syn_sentence, y_true):
  differences = []
  for idx in range(len(y_true)):
    y_hat_syn = y_syn_sentence[idx][y_true[idx]]
    y_hat = y_org_sentence[idx][y_true[idx]]
    # print(y_hat_syn)
    # print(y_hat)
    diff = abs(y_hat - y_hat_syn)
    differences.append(diff)
  metric_mean =  sum(differences)/len(y_pred) #
  metric_median = statistics.median(differences)
  return metric_mean, metric_median
metric_mean, metric_median = evaluate_embeddings(y_org_sentence, y_syn_sentence, y_true)

In [ ]:
metric_mean

tensor(0.0907, grad_fn=<DivBackward0>)

In [ ]:
metric_median

tensor(0.0267, grad_fn=<DivBackward0>)